# Análisis del modelo de intenciones (TF-IDF + Logistic Regression)

Este cuaderno evalúa el clasificador supervisado de intenciones usando el dataset de ejemplo y muestra métricas básicas, matriz de confusión, curva de aprendizaje y características más influyentes por clase.

In [ ]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, learning_curve
import matplotlib.pyplot as plt

# Asegurar acceso al paquete de la app
ROOT = Path().resolve().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from app.ml.intent import build_pipeline
from app.ml.keywords import classify_text

data_path = ROOT / 'app' / 'ml' / 'data' / 'intent_samples.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
# Train/test split y entrenamiento del pipeline
X = df['text'].astype(str).values
y = df['label'].astype(str).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
pipe = build_pipeline()
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.3f}')
print(classification_report(y_test, y_pred))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred, labels=pipe.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipe.classes_)
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Curva de aprendizaje
train_sizes, train_scores, valid_scores = learning_curve(
    estimator=pipe,
    X=X,
    y=y,
    train_sizes=np.linspace(0.2, 1.0, 5),
    cv=3,
    scoring='accuracy',
    shuffle=True,
    random_state=42
)
train_mean = train_scores.mean(axis=1)
valid_mean = valid_scores.mean(axis=1)
plt.figure(figsize=(6,4))
plt.plot(train_sizes, train_mean, 'o-', label='train')
plt.plot(train_sizes, valid_mean, 'o-', label='cv')
plt.xlabel('Training examples'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

In [ ]:
# Características más influyentes por clase (coeficientes)
vect = pipe.named_steps['tfidf']
clf = pipe.named_steps['clf']
feature_names = np.array(vect.get_feature_names_out())

def top_features_for(label: str, k: int = 10):
    idx = list(clf.classes_).index(label)
    coefs = clf.coef_[idx]
    topk = np.argsort(coefs)[-k:][::-1]
    return list(zip(feature_names[topk].tolist(), np.round(coefs[topk], 3).tolist()))

for label in clf.classes_:
    print(label, '->', top_features_for(label, 8))

In [ ]:
# Comparación rápida: supervisado vs. palabras clave
samples = [
    'Quiero agendar un turno para mi perro',
    'Necesito el refuerzo de la vacuna rabia',
    'Es una emergencia, no respira',
    'Cuál es el precio de la castración?',
    'Qué horarios tienen los domingos?',
    'Me pasan el WhatsApp?',
    'Dónde están ubicados?',
    'Busco peluquería canina y baño',
    'Hola, quería consultar'
]
for s in samples:
    sup_label = pipe.predict([s])[0]
    sup_proba = pipe.predict_proba([s])[0].max()
    kw_label, kw_kws, kw_conf = classify_text(s)
    print(f'- {s}\n  supervised: {sup_label} ({sup_proba:.2f}) | keywords: {kw_label} ({kw_conf:.2f}) -> kws: {kw_kws}')

## Notas
- El dataset es pequeño y de ejemplo; ampliar con frases reales mejora precisión.
- Se puede probar class_weight='balanced' en LogisticRegression si hay desbalance.
- Guardar el pipeline entrenado con app/ml/intent.py y probar el endpoint /ai/intent.